In [2]:
import os
os.environ['TRANSFORMERS_CACHE'] = "/media/data/flowers/OpenELM/models"
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "Salesforce/codegen-350M-mono" # mono is better
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /media/data/flowers/conda/envs/codegpt/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /media/data/flowers/conda/envs/codegpt/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /media/data/flowers/conda/envs/codegpt/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


In [3]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [4]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [5]:
model

CodeGenForCausalLM(
  (transformer): CodeGenModel(
    (wte): Embedding(51200, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-19): 20 x CodeGenBlock(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): CodeGenAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (qkv_proj): Linear4bit(in_features=1024, out_features=3072, bias=False)
          (out_proj): Linear4bit(in_features=1024, out_features=1024, bias=False)
        )
        (mlp): CodeGenMLP(
          (fc_in): Linear4bit(in_features=1024, out_features=4096, bias=True)
          (fc_out): Linear4bit(in_features=4096, out_features=1024, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=51200, bi

In [6]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["qkv_proj"],#["query_key_value"], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 655360 || all params: 231538688 || trainable%: 0.2830455703368242


In [7]:
from datasets import load_dataset
path= "/media/data/flowers/OpenELM/logs/elm/23-08-16_16:29/step_9/save_all.json"
squad_it_dataset = load_dataset("json", data_files=path)


In [8]:
path_test = "/media/data/flowers/OpenELM/preprocess_p3.json" # trainset from P3
test_set = load_dataset("json", data_files=path_test)

In [9]:
def filter_correct_puzz(example):
    return (
        example["fitness"] >0.5
    )


In [10]:
correct_dataset = squad_it_dataset.filter(filter_correct_puzz)


In [11]:
# def tokenize(element):
#     context_length=2048
#     outputs = tokenizer(
#         element["program_str"],
#         truncation=True,
#         max_length=context_length,
#         # return_length=True,
#     )
#     input_batch = []
#     input_ids = outputs["input_ids"]
#     input_batch.append(input_ids)
#     # for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
#     #     if length == context_length:
#     #         input_batch.append(input_ids)
#     return {"input_ids": input_batch}


# tokenized_datasets = correct_dataset.map(
#     tokenize, batched=True, remove_columns=correct_dataset["train"].column_names
# )
tokenized_datasets = correct_dataset.map(lambda samples: tokenizer(samples["program_str"]), batched=True)
tok_test_set = test_set.map(lambda samples: tokenizer(samples["program_str"]), batched=True)

Map: 100%|██████████| 536/536 [00:00<00:00, 11610.23 examples/s]


In [12]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_datasets["train"],
    eval_dataset = tok_test_set,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        evaluation_strategy="epoch", #"steps",
        gradient_accumulation_steps=1,
        # warmup_steps=2,
        warmup_ratio=0.1,
        num_train_epochs = 5,
        # max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: julien_p (flowers-team). Use `wandb login --relogin` to force relogin


  2%|▏         | 1/50 [00:00<00:44,  1.11it/s]

{'loss': 1.3192, 'learning_rate': 4e-05, 'epoch': 0.1}


  4%|▍         | 2/50 [00:01<00:29,  1.65it/s]

{'loss': 1.2797, 'learning_rate': 8e-05, 'epoch': 0.2}


...

100%|██████████| 50/50 [01:19<00:00,  1.65it/s]

{'loss': 0.4062, 'learning_rate': 4.444444444444445e-06, 'epoch': 4.9}
{'loss': 0.2851, 'learning_rate': 0.0, 'epoch': 5.0}


                                               
100%|██████████| 50/50 [01:33<00:00,  1.86s/it]

{'eval_train_loss': 1.5257525444030762, 'eval_train_runtime': 13.205, 'eval_train_samples_per_second': 40.591, 'eval_train_steps_per_second': 5.074, 'epoch': 5.0}
{'train_runtime': 100.1006, 'train_samples_per_second': 3.696, 'train_steps_per_second': 0.499, 'train_loss': 0.6666267824172973, 'epoch': 5.0}


TrainOutput(global_step=50, training_loss=0.6666267824172973, metrics={'train_runtime': 100.1006, 'train_samples_per_second': 3.696, 'train_steps_per_second': 0.499, 'train_loss': 0.6666267824172973, 'epoch': 5.0})